In [1]:
# In here we sort and only leave the questions that students answer right more than 80% or less than 20%.
# These questions are considered "not making sense questions"
import csv
import time
import json
import pandas as pd


#create dictionary key=Question ID, value=Max_points_count
def create_ids_dict_max_points(input_file):
    with open(input_file, mode='r', encoding="utf8" ) as f:
        id_dict={}
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            cnt=0
            key=str(row["ID вопроса"])
            key2=int(row["максимальный балл за этот вопрос"])
            if key in id_dict:
                #update count for ID
                cnt=int(id_dict[key])+key2
                id_dict.update({key : cnt})
            else:
                #insert new ID
                id_dict.update([(key,key2)] ) 
    return id_dict

#create dictionary key=Question ID, value=right_answers_count
def create_ids_dict_right_answers(input_file):
    with open(input_file, mode='r', encoding="utf8" ) as f:
        id_dict={}
        reader = csv.DictReader(f, delimiter=',')
        for row in reader:
            cnt=0
            key=str(row["ID вопроса"])
            key2=int(row["правильность ответа ученика"])
            if key in id_dict:
                #update count for ID
                cnt=int(id_dict[key])+key2
                id_dict.update({key : cnt})
            else:
                #insert new ID
                id_dict.update([(key,key2)] ) 
    return id_dict

def get_fields_nm(input_file):
    with open(input_file, "r",encoding="utf8") as f:
        reader = csv.reader(f)
        fields_nm=next(reader)
        return fields_nm


def write_cleaned_data(input_file,output_file,field_names,filter_nm):
    with open(input_file, mode='r',encoding="utf8") as f, open(output_file, 'w',encoding="utf8") as csvfile :
        reader=csv.DictReader(f, delimiter=',')
        writer=csv.DictWriter(csvfile,delimiter=',', fieldnames=field_names)
        writer.writeheader()
        for row in reader:
            if row["ID вопроса"] in filter_nm:
                writer.writerow(row)

if __name__ == "__main__":        
    start_time = time.time()
    file_nm='Cleared_data.csv'
    output_file_nm='wrong_questions_ratio.csv'
    csv_fieldnames=get_fields_nm(file_nm)
    id_dictionary_max_points=create_ids_dict_max_points(file_nm)
    print("Create id_dictionary for Max points took {} seconds".format( time.time() - start_time   ) )
    start_time1 = time.time()
    id_dictionary_right_answers=create_ids_dict_right_answers(file_nm)
    
    print("Create id_dictionary for right answers took {} seconds".format( time.time() - start_time1   ) )
    #print(id_dictionary_max_points)
    #print('\\n')
    #print(id_dictionary_right_answers)
    id_dictionary_ratio = {}
    for key in id_dictionary_max_points:
        x = id_dictionary_right_answers[key]
        y = id_dictionary_max_points[key]
        z = x/y
        id_dictionary_ratio[key] = z
    
    
    dict_filetered = {k:v for (k,v) in id_dictionary_ratio.items() if v<=0.2 or v>=0.8}
    #save filtered id into file
    with open('filtered_ids.json', 'w') as fp:
        json.dump(dict_filetered, fp)
    start_time2 = time.time()
    write_cleaned_data(file_nm,output_file_nm,csv_fieldnames,dict_filetered)
    print("Write to CSV file took {} seconds".format( time.time() - start_time2   ) )

Create id_dictionary for Max points took 215.8954677581787 seconds
Create id_dictionary for right answers took 215.22696781158447 seconds
